## Zadanie domowe: BBHE i DSIHE

W klasycznym wyrównywaniu histogramu HE  po wykonaniu operacji jasność obrazu ulega zmianie.
Dało się to zaobserwować podczas przeprowadzonych eksperymentów.
Jeśli nie to należy uruchomić skrypt z sekcji A i zwrócić na to uwagę.
Średnia jasność dąży do środkowego poziomu szarości.
Jest to wada i dlatego klasyczne HE ma ograniczone zastosowanie.

Powstało sporo metod, które eliminują to niekorzystne zjawisko.
Najprostsze z nich polegają na dekompozycji obrazu wejściowego na dwa podobrazy (wg. pewnego kryterium).
Następnie operacja HE wykonywana jest dla tych podobrazów.

Dwie znane z literatury metody to:
- Bi-Histogram Equalization
- DSIHE - Dualistic Sub-Image Histogram Equalization

W metodzie BBHE za kryterium podziału przyjmuje się średnią jasność w obrazie.
W DSIHE obraz dzieli się na dwa podobrazy o takiej samej liczbie pikseli (jaśniejszych i ciemniejszych).

W ramach zadania należy zaimplementować wybraną metodę: BBHE lub DSIHE (ew. obie).

1. Wczytaj obraz *jet.bmp* i wylicz jego histogram.
2. W kolejnym kroku należy wyznaczyć próg podziału obrazu na dwa podobrazy (*lm*).
3. Dla BBHE wyznacz średnią jasność obrazu. Dla DSIHE można wykorzystać histogram skumulowany.
Należy znaleźć poziom jasności który znajduje się "w połowie" histogramu skumulowanego.
W tym celu warto stworzyć tablicę, zawierającą moduł histogramu skumulowanego pomniejszonego o połowę liczby pikseli.
Następnie znaleźć minimum.
4. Dalej należy podzielić histogram oryginalnego obrazu na dwa histogramy *H1* i *H2*.
Dla każdego z nich wyliczyć histogram skumulowany ($C_1$ i $C_2$) i wykonać normalizację.
Normalizacja polega na podzieleniu każdego histogramu przez jego największy element.
5. Na podstawie histogramów skumulowanych należy stworzyć przekształcenie LUT.
Należy tak przeskalować $C_1$ i $C_2$, aby uzyskać jednorodne przekształcenie.
Tablicę $C_1$ wystarczy pomnożyć przez próg podziału.
Tablicę $C_2$ należy przeskalować do przedziału: $<lm+1; 255>$, gdzie $lm$ jest progiem podziału.<br>
$C_{1n} = (lm)*C1;$<br>
$C_{2n} = lm+1 + (255-lm-1)*C2;$<br>
Następnie dwie części tablicy przekodowań należy połączyć.
6. Ostatecznie należy wykonać operację LUT i wyświetlić wynik wyrównywania histogramu.
Porównaj wynik operacji BBHE lub DSIHE z klasycznym HE.

In [ ]:
import cv2
import os
from matplotlib import pyplot as plt
import numpy as np

if not os.path.exists("jet.bmp") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/03_Histogram/jet.bmp --no-check-certificate

#1. Wczytanie obrazu i wyliczwnie histogramu
img = cv2.imread('jet.bmp', cv2.IMREAD_GRAYSCALE)
img_hist = cv2.calcHist([img], [0], None, [256], [0, 256])

#2. Wyznaczenie progu podziału obrazu na dwa podobrazy (lm)
hist_cum = img_hist.cumsum()
half_pixels = img.size // 2 #liczba pikseli w połowie histogramu skumulowanego
lm = np.abs(hist_cum - half_pixels).argmin() #poziom jasności w połowie histogramu skumulowanego

#3. Histogram skumulowany DSIHE
hist_cum_dsihe = img_hist.cumsum()

#4. Podział histogramu oryginalnego obrazu na dwa histogramy H1 i H2 i ich normalizacja
H1 = img_hist[: lm + 1]
H2 = img_hist[lm + 1 :]
C1 = H1.cumsum()
C2 = H2.cumsum()
C1n = cv2.normalize(C1, None, 0, 255, cv2.NORM_MINMAX)
C2n = cv2.normalize(C2, None, 0, 255, cv2.NORM_MINMAX)

#5. Tablica LUT
C1n_scal = lm * C1n
C2n_scal = lm + 1 + (255 - lm - 1) * C2n
lut = np.concatenate((C1n_scal, C2n_scal))
lut_norm = cv2.normalize(lut, None, 0, 255, cv2.NORM_MINMAX)
lut_dsihe = cv2.LUT(img, lut_norm)

#6. Wykonanie operacji LUT i wyświetlenie wyniku wyrównania histogramu oraz porównanie z klasycznym HE
img_eq = cv2.LUT(img, lut.astype('uint8'))
plt.subplot(1, 2, 1)
plt.imshow(img, cmap = 'gray')
plt.title('Original Image')
plt.subplot(1, 2, 2)
plt.imshow(img_eq, cmap = 'gray')
plt.title('DSIHE Result')
plt.show()

img_eq2 = cv2.equalizeHist(img)
plt.subplot(1, 2, 1)
plt.imshow(img_eq, cmap = 'gray')
plt.title('DSIHE Result')
plt.subplot(1, 2, 2)
plt.imshow(img_eq2, cmap = 'gray') 
plt.title('Classic HE Result')
plt.show()